In [1]:
import subprocess
import cv2
from subprocess import check_output
import numpy as np
from scipy import spatial
import time
import _thread

from transform.calibration import *
from pdf.pdf2img import PdfFile

In [2]:
## parameters
proj_h = 720
proj_w = 1280

## init settings
circles = []
refPt = []
warp = False
stopReading = False

Code structure-

Read Book:
    Convert PDF to Images
    Show page 1 (with transformation)
    while (!stopReading) :
        pageChange = Read Output Sensor
        if pageChange == nextPage :
            Read Page(+1)
        else if pageChange == prevPage :
            Read Page(-1)

In [3]:
def get_num_pages(pdf_path):
    output = check_output(["pdfinfo", pdf_path]).decode()
    pages_line = [line for line in output.splitlines() if "Pages:" in line][0]
    num_pages = int(pages_line.split(":")[1])
    return num_pages

In [4]:
def ConvertPdfToImages (pdfFile, imageFile) :
    
    # Path to Poppler's libraries (Download from https://blog.alivate.com.au/poppler-windows/)
    pdfToPPMPath = r"C:\Program Files (x86)\Poppler\poppler-0.68.0\bin\pdftoppm.exe"

    # Convert PDF to images
    subprocess.Popen('"%s" -png %s %s' % (pdfToPPMPath, pdfFile, imageFile))
    
    # Return number of pages in PDF
    return get_num_pages(pdfFile)

In [5]:
def imgResize(img):
#     img = cv2.imread(imageFile)
    height, width = img.shape[:2]
    # resize img
    ratio = max(width/proj_w, height/proj_h);
    img = cv2.resize(img, (int(width/ratio), int(height/ratio)))
    return img

def pageShow (img, mat) :
#     filename = imageFile + '-' + str(pageNumber) + '.jpg'
#     img = ReadImg(filename)
    # Open CV code for transformation
    img = cv2.warpPerspective(img, mat, (proj_w, proj_h))
    cv2.imshow(fr_trans, img)
    key = cv2.waitKey(1)
    return key
    
def initFrame(frameName):
    cv2.namedWindow(frameName)


In [6]:
def ReadSensorOutput (threadName) :
    global page_action, detectPage
    while True:
        time.sleep(1)
        if detectPage == True:
            page_action.append(1)
        print("move forward ", page_action)
    # Return 1 for next page, -1 for prev page based on threshold
    
def ReadKeyboardInput ():
    key = []
    while True:
        key = cv2.waitKey(0)
    print("key, ", key)
    if key == 97: #'a'
        pageNum = pageNum-1
        img_next = file.readPage(pageNum)
        print ("detect a")
    elif key == 100: #'d'
        pageNum = pageNum+1
        img_next = file.readPage(pageNum)
        print ("detect d")
    

In [7]:
def ReadBook (pdfFile) :
    global stopReading, fr_trans, page_action, detectPage
    detectPage = False
    page_action = []
    file = PdfFile()
    totalPages = file.readFile(pdfFile)
    print("total: ", totalPages)
    if totalPages < 1:
        return -1
    pageNumber = 0
    img = file.readPage(pageNumber)
    img = imgResize(img)
    # calibration
    mat = np.identity(3)
    mat = calibration(img)
    print("img shape: ", img.shape)
    pageShow(img, mat)
    try:
        _thread.start_new_thread( ReadSensorOutput, ("Thread-1",))
    except:
        print ("Error: unable to start thread")
    pageChange = 0
    while (stopReading == False) : # Replace with a listener to signal end of reading
        detectPage = True
        pageChange = 0
        if len(page_action)>0:
            pageChange = page_action[-1]
            print("get page change ", pageChange)
            del page_action[-1]
        time.sleep(0.5)
#         pageChange = ReadSensorOutput()
#         pageChange = ReadKeyboardInput()
        key = pageShow(img, mat)
        if key == 27:
            print("stop")
            break
        elif key == 97: #'a'
            pageChange = 1
            print ("detect a")
        elif key == 100: #'d'
            pageChange = -1
            print ("detect d")
#         page changing    
        if pageChange==1 and pageNumber<totalPages-1 : 
            pageNumber = pageNumber + 1
            print(pageNumber)
            img = imgResize(file.readPage(pageNumber))
        elif pageChange==-1 and pageNumber>0 :
            pageNumber = pageNumber - 1
            img = imgResize(file.readPage(pageNumber))
    detectPage = False
#     cv2.destroyWindow(fr_trans)


In [8]:
def print_time( threadName):
       delay = 1
       count = 0
       while count < 5:
          time.sleep(delay)
          count += 1
          print ("%s: %s" % ( threadName, time.ctime(time.time()) ))

In [9]:
def readNameFromFolder(folderName):
    import os
    pdfFiles = []
    for file in os.listdir(folderName):
        if file.endswith(".pdf"):
            pdfFiles.append(os.path.join(folderName, file))
            print(os.path.join(folderName, file))
    return pdfFiles

In [ ]:
if __name__ == "__main__":
#     pdfFile = "pdf/test1.pdf"
    detectPage = False
    page_action = []
    folderName = "pdf/"
    pdfFiles = readNameFromFolder(folderName)
    imgBlack = np.zeros((proj_h, proj_w))
    if pdfFiles != []:
        fr_trans = "Perspective transformation"
        initFrame(fr_trans)
        print("Please select a file!")
        fileIdx = 0
        def IdxChange(idx, action, numMax, minMax):
            result = idx + action
            if result >= minMax and result <= numMax:
                idx = result
            else:
                result = idx
            return result
        while True:
            cv2.imshow(fr_trans, imgBlack)
            key = cv2.waitKey(1)
            if key == 27:
                print("stop")
                break
            elif key == 13:
                print("Enter")
                print("read file: ", pdfFiles[fileIdx])
                ReadBook (pdfFiles[fileIdx])
            elif key == 97: #'a'
                fileIdx = IdxChange(fileIdx, -1, 0, len(pdfFiles)-1)
                print ("detect a")
            elif key == 100: #'d'
                fileIdx = IdxChange(fileIdx, +1, 0, len(pdfFiles)-1)
                print ("detect d")


    
    cv2.destroyAllWindows()
    
    


pdf/lab2.pdf
pdf/test1.pdf
Please select a file!
Enter
read file:  pdf/lab2.pdf
total:  3
warp
pos:  [647 136] . nearest!  3
warp
pos:  [682 511] . nearest!  2
warp
pos:  [660 154] . nearest!  3
stop
img shape:  (720, 556, 3)
move forward  [1]
get page change  1
1
move forward  [1]
detect d
get page change  1
move forward  [1]
1
get page change  1
2
move forward  [1]
get page change  1
move forward  [1]
get page change  1
move forward  [1]
get page change  1
move forward  [1]
get page change  1
move forward  [1]
get page change  1
move forward  [1]
get page change  1
move forward  [1]
stop
move forward  [1, 1]
move forward  [1, 1, 1]
move forward  [1, 1, 1, 1]
detect d
move forward  [1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Enter
read file:  pdf/lab2.pdf
move forward  []
m

move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
move forward  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
_thread.start_new_thread( ReadSensorOutput, ("Thread-1"))

In [ ]:
page_action
